In [1]:
import os
import torch
import numpy as np
import pandas as pd
import sys
import random
import h5py

sys.path.append('../')

from src import *
from src.crowd_count import *
from src.network import *
from src.data_loader import *
from src.timer import *
from src.evaluate_model import *

## Loading in the NN

In [2]:
# load net
mcnn = CrowdCounter('mcnn')
weights_normal_init(mcnn, dev=0.01) # wtheck does this do

## Loading in the data

In [3]:
# specify path to the consolidated .h5 files
train_path = '../data/SHT/part_A_final/train_data/consolidated'
val_path = '../data/SHT/part_A_final/test_data/consolidated'

data_loader = ImageDataLoader(train_path, shuffle=False, pre_load=False)
data_loader_val = ImageDataLoader(val_path, shuffle=False, pre_load=False)

## Training

In [4]:
# specifying the model
model = mcnn
model.train(True)

# specifying some settings
output_dir = '../output/{}'.format(model.name)
dataset_name = 'SHT_A'
best_mae, best_mse, best_epoch = 999999, 999999, 0

# train on the GPU or on the CPU, if a GPU is not available
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

# construct an optimizer
params = [p for p in model.parameters() if p.requires_grad]
#optimizer = torch.optim.SGD(params, lr=0.005, momentum=0.9, weight_decay=0.0005)
optimizer = torch.optim.Adam(params, lr=0.00001)
    
# learning rate scheduler
lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=3, gamma=0.1)

# set the number of epochs
num_epochs = 2

for epoch in range(num_epochs):
    #model.train(True)
    train_loss = 0
    for train in data_loader:
        im_data = train['data']
        gt_data = train['gt_density']
        
        density_map = model(im_data, gt_data)

        loss = model.loss
        train_loss += loss.data
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
    # update the learning rate
    lr_scheduler.step()
    
    # evaluate on the validation dataset
    #model.train(False)
    mae, mse = 0, 0
    with torch.no_grad():
        for val in data_loader_val:
            im_data_val = val['data']
            gt_data_val = val['gt_density']
            dmap = model(im_data_val, gt_data_val).data.cpu().numpy()

            gt_count = gt_data_val.sum()
            et_count = np.sum(dmap)

            mae += abs(gt_count - et_count)
            mse += (gt_count-et_count)**2

        mae /= data_loader_val.get_num_samples()
        mse /= data_loader_val.get_num_samples()
    
    # want to save the best model as at the current epoch
    if (mae < best_mae) and (mse < best_mse):
        save_to = os.path.join(output_dir, '{}_{}_best.h5'.format(model.name, dataset_name))
        save_net(save_to, model)
        best_epoch = epoch
        best_mae = mae
        best_mse = mse
    
    print('epoch: {}, MAE: {}, MSE: {}'.format(epoch, round(mae, 2), round(mse, 2)))
    
print()
print('------BEST MODEL------')
print('epoch: {}, MAE: {}, MSE: {}'.format(best_epoch, round(best_mae, 2), round(best_mse, 2)))
print('model saved to: {}'.format(output_dir))

epoch: 0, MAE: 504.62, MSE: 421711.76
epoch: 1, MAE: 504.28, MSE: 421335.32

------BEST MODEL------
epoch: 1, MAE: 504.28, MSE: 421335.32
model saved to: ../output/mcnn
